In [ ]:
import cv2
import numpy as np
import os

# 设置输入图像的路径
image_path = r'C:\Users\70894\Desktop\LatLRR\IR\18.png'
output_mask_folder = r'C:\Users\70894\Desktop\LatLRR\mask'
output_image_folder = r'C:\Users\70894\Desktop\LatLRR\mask_image'

# 确保输出文件夹存在
os.makedirs(output_mask_folder, exist_ok=True)
os.makedirs(output_image_folder, exist_ok=True)

# 读取图像
image = cv2.imread(image_path)
if image is None:
    raise ValueError(f"Cannot load image from path {image_path}")

# 初始化掩码、背景模型和前景模型
final_mask = np.zeros(image.shape[:2], np.uint8)  # 用于存储所有选择的掩码
bgd_model = np.zeros((1, 65), np.float64)
fgd_model = np.zeros((1, 65), np.float64)

while True:
    # 用户定义一个初始的矩形ROI
    rect = cv2.selectROI(f"Select ROI for {os.path.basename(image_path)} (press Enter to confirm, Esc to finish)", image, fromCenter=False, showCrosshair=True)
    
    # 如果用户按下 Esc 键，退出ROI选择
    if rect[2] == 0 or rect[3] == 0:  # 如果没有选择任何区域，则退出
        break
    
    # 初始化当前掩码
    mask = np.zeros(image.shape[:2], np.uint8)
    
    # 应用GrabCut算法来细化分割显著目标（迭代次数设为15）
    cv2.grabCut(image, mask, rect, bgd_model, fgd_model, 15, cv2.GC_INIT_WITH_RECT)
    
    # 将处理后的掩码转换为二值掩码
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    
    # 使用高斯模糊平滑掩码
    mask2 = cv2.GaussianBlur(mask2, (5, 5), 0)
    
    # 膨胀和腐蚀操作，进一步平滑边缘
    kernel = np.ones((3, 3), np.uint8)
    mask2 = cv2.dilate(mask2, kernel, iterations=1)  # 膨胀操作
    mask2 = cv2.erode(mask2, kernel, iterations=1)   # 腐蚀操作

    # 将当前掩码叠加到最终掩码上
    final_mask = cv2.bitwise_or(final_mask, mask2)

    # 显示当前选择的区域
    masked_image = image * final_mask[:, :, np.newaxis]
    cv2.imshow("Masked Image", masked_image)

    # 按 Enter 键继续选择，按 Esc 键退出
    key = cv2.waitKey(0)
    if key == 27:  # 按下 'Esc' 键，退出循环
        break

# 应用最终的掩码到图像
masked_image = image * final_mask[:, :, np.newaxis]

# 构造输出文件路径
mask_output_path = os.path.join(output_mask_folder, f"mask_{os.path.basename(image_path)}")
masked_image_output_path = os.path.join(output_image_folder, f"masked_{os.path.basename(image_path)}")

# 保存二值掩码图像
cv2.imwrite(mask_output_path, final_mask * 255)

# 保存应用了掩码的显著目标图像
cv2.imwrite(masked_image_output_path, masked_image)

# 关闭所有窗口
cv2.destroyAllWindows()

print(f"Processed {os.path.basename(image_path)}, saved mask to {mask_output_path}, saved masked image to {masked_image_output_path}")


In [1]:
# 图像融合
import numpy as np
from scipy.linalg import svd, inv
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import os

def latent_lrr(X, lambda_val, tol=1e-6, rho=1.1, max_mu=1e6, mu=1e-6, max_iter=1000):
    A = X
    d, n = X.shape
    m = n
    
    # Initialize variables
    J = np.zeros((m, n))
    Z = np.zeros((m, n))
    L = np.zeros((d, d))
    S = np.zeros((d, d))
    E = np.zeros((d, n))
    
    Y1 = np.zeros((d, n))
    Y2 = np.zeros((m, n))
    Y3 = np.zeros((d, d))
    
    atx = np.dot(X.T, X)
    inv_a = inv(np.dot(A.T, A) + np.eye(m))
    inv_b = inv(np.dot(A, A.T) + np.eye(d))
    
    iter = 0
    while iter < max_iter:
        iter += 1
        
        # Update J using Singular Value Thresholding (SVT)
        temp_J = Z + Y2 / mu
        U_J, sigma_J, V_J = svd(temp_J, full_matrices=False)
        svp_J = len(np.where(sigma_J > 1 / mu)[0])
        if svp_J > 0:
            sigma_J = sigma_J[:svp_J] - 1 / mu
        else:
            svp_J = 1
            sigma_J = np.array([0])
        J = np.dot(U_J[:, :svp_J], np.dot(np.diag(sigma_J), V_J[:svp_J, :]))
        
        # Update S using Singular Value Thresholding (SVT)
        temp_S = L + Y3 / mu
        U_S, sigma_S, V_S = svd(temp_S, full_matrices=False)
        svp_S = len(np.where(sigma_S > 1 / mu)[0])
        if svp_S > 0:
            sigma_S = sigma_S[:svp_S] - 1 / mu
        else:
            svp_S = 1
            sigma_S = np.array([0])
        S = np.dot(U_S[:, :svp_S], np.dot(np.diag(sigma_S), V_S[:svp_S, :]))
        
        # Update Z
        Z = np.dot(inv_a, atx - np.dot(X.T, np.dot(L, X)) - np.dot(X.T, E) + J + (np.dot(X.T, Y1) - Y2) / mu)
        
        # Update L
        L = np.dot(((X - np.dot(X, Z) - E).dot(X.T) + S + (np.dot(Y1, X.T) - Y3) / mu), inv_b)
        
        # Update E
        xmaz = X - np.dot(X, Z) - np.dot(L, X)
        temp = xmaz + Y1 / mu
        E = np.maximum(0, temp - lambda_val / mu) + np.minimum(0, temp + lambda_val / mu)
        
        # Check convergence
        leq1 = xmaz - E
        leq2 = Z - J
        leq3 = L - S
        stopC = max(np.max(np.abs(leq1)), np.max(np.abs(leq2)), np.max(np.abs(leq3)))
        
        if stopC < tol:
            print('LRR done.')
            break
        else:
            Y1 += mu * leq1
            Y2 += mu * leq2
            Y3 += mu * leq3
            mu = min(max_mu, mu * rho)
    
    return Z, L, E

# 设置路径
ir_path = r'C:\Users\70894\Desktop\LatLRR\IR'
vis_path = r'C:\Users\70894\Desktop\LatLRR\VIS'
mask_path = r'C:\Users\70894\Desktop\LatLRR\mask'
output_path = r'C:\Users\70894\Desktop\LatLRR\Fused'

# 获取所有图像文件
ir_files = sorted(os.listdir(ir_path))
vis_files = sorted(os.listdir(vis_path))

# 逐一处理图像
for ir_file, vis_file in zip(ir_files, vis_files):
    # 加载图像
    ir_image = Image.open(os.path.join(ir_path, ir_file))
    vis_image = Image.open(os.path.join(vis_path, vis_file))
    mask_file = f"mask_{ir_file}"  # 假设掩码文件命名为"mask_红外图像名.jpg"
    mask_image = Image.open(os.path.join(mask_path, mask_file))
    
    # 获取图像的原始尺寸
    image_size = ir_image.size
    
    # 将掩码调整为与图像相同的尺寸
    mask_image = mask_image.resize(image_size, Image.BILINEAR)
    
    # 转换为灰度图像并转换为numpy数组
    ir_image = np.array(ir_image.convert('L'), dtype=float)
    vis_image = np.array(vis_image.convert('L'), dtype=float)
    mask_image = np.array(mask_image) / 255.0  # 归一化到 [0, 1]
    
    # 归一化处理
    ir_image = ir_image / 255.0
    vis_image = vis_image / 255.0
    
    # 将图像数据展开成矩阵
    ir_flat = ir_image.reshape(-1, ir_image.shape[1])
    vis_flat = vis_image.reshape(-1, vis_image.shape[1])
    
    lambda_val = 0.1
    
    # 对两张图像分别进行潜在低秩分解
    Z1, L1, E1 = latent_lrr(ir_flat, lambda_val)
    Z2, L2, E2 = latent_lrr(vis_flat, lambda_val)
    
    # 计算显著性部分和低秩部分
    I_lrr1 = np.dot(ir_flat, Z1)
    I_saliency1 = np.dot(L1, ir_flat)
    I_lrr1 = np.clip(I_lrr1, 0, 1)
    I_saliency1 = np.clip(I_saliency1, 0, 1)
    
    I_lrr2 = np.dot(vis_flat, Z2)
    I_saliency2 = np.dot(L2, vis_flat)
    I_lrr2 = np.clip(I_lrr2, 0, 1)
    I_saliency2 = np.clip(I_saliency2, 0, 1)
      
    # 在背景区域进行融合，确保背景亮度提升
    F_lrr_background = (I_lrr1*0.5 + I_lrr2*0.5)  # 融合红外和可见光的低秩部分
    F_saliency_background = I_saliency1*0.5 + I_saliency2*0.5 # 融合显著性部分
    
    # 在显著区域进行加权融合，保留红外和可见光信息
    alpha = 0.9 # 可根据需求调整权重
    F_lrr = mask_image * (alpha * I_lrr1 + (1 - alpha) * I_lrr2) + (1 - mask_image) * F_lrr_background
    F_saliency = mask_image * (alpha * I_saliency1 + (1 - alpha) * I_saliency2) + (1 - mask_image) * F_saliency_background
    
    # 重构融合后的图像
    F = F_lrr + F_saliency
    F = np.clip(F, 0, 1)
    
    # 重构图像转换回原始形状
    F_reconstructed = F.reshape(ir_image.shape)
    
    # 使用源图像的名称保存融合后的图像
    output_file = os.path.join(output_path, ir_file)
    cv2.imwrite(output_file, (F_reconstructed * 255).astype(np.uint8))
    
    print(f"Fused image saved at {output_file}")
    
   

    # # 显示源图像和融合图像
    # plt.figure(figsize=(12, 8))
    
    # plt.subplot(1, 3, 1)
    # plt.title(f'IR Image {i+1}')
    # plt.imshow(ir_image, cmap='gray')
    
    # plt.subplot(1, 3, 2)
    # plt.title(f'VIS Image {i+1}')
    # plt.imshow(vis_image, cmap='gray')
    
    # plt.subplot(1, 3, 3)
    # plt.title(f'Fused Image {i+1}')
    # plt.imshow(F_reconstructed, cmap='gray')
    
    # plt.show()


LRR done.
LRR done.
Fused image saved at C:\Users\70894\Desktop\LatLRR\Fused\01.png
LRR done.
LRR done.
Fused image saved at C:\Users\70894\Desktop\LatLRR\Fused\02.png
LRR done.
LRR done.
Fused image saved at C:\Users\70894\Desktop\LatLRR\Fused\03.png


In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio, mean_squared_error
from scipy.stats import entropy

# 读取图像
def read_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    return image

# 计算熵（Entropy）
def entropy_evaluation(image):
    hist, _ = np.histogram(image.ravel(), bins=256, range=(0, 256))
    return entropy(hist, base=2)

# 计算均方误差（MSE）
def mse_evaluation(imageA, imageB, fused_image):
    mse_A = mean_squared_error(imageA, fused_image)
    mse_B = mean_squared_error(imageB, fused_image)
    return 0.5 * (mse_A + mse_B)

# 计算峰值信噪比（PSNR）
def psnr_evaluation(imageA, imageB, fused_image):
    mse_A = mean_squared_error(imageA, fused_image)
    mse_B = mean_squared_error(imageB, fused_image)
    mse_avg = 0.5 * (mse_A + mse_B)
    return 20 * np.log10(255 / np.sqrt(mse_avg))

# 计算结构相似性（SSIM）
def ssim_evaluation(imageA, imageB, fused_image):
    ssim_A = ssim(imageA, fused_image)
    ssim_B = ssim(imageB, fused_image)
    return 0.5 * (ssim_A + ssim_B)

# 计算互信息（MI）
def mutual_information(imageA, imageB, fused_image, grey_level=256):
    def joint_histogram(a, b, bins):
        hist_2d, _, _ = np.histogram2d(a.ravel(), b.ravel(), bins=bins)
        return hist_2d

    HA = entropy_evaluation(imageA)
    HB = entropy_evaluation(imageB)
    HF = entropy_evaluation(fused_image)
    
    HFA = entropy(joint_histogram(fused_image, imageA, grey_level))
    HFB = entropy(joint_histogram(fused_image, imageB, grey_level))
    
    MIFA = HA + HF - HFA
    MIFB = HB + HF - HFB
    return MIFA + MIFB

# 主函数，计算所有指标
def analysis_reference(fused_image, imageA, imageB, easy):
    # 计算 EN 和 MI 需要 uint8 数据，其他转换为浮点型
    fused_image = fused_image.astype(np.uint8)
    imageA = imageA.astype(np.uint8)
    imageB = imageB.astype(np.uint8)

    EN = entropy_evaluation(fused_image)
    MI = mutual_information(imageA, imageB, fused_image)

    # 转换为浮点型用于计算 PSNR, MSE 等
    fused_image_float = fused_image.astype(np.float64)
    imageA_float = imageA.astype(np.float64)
    imageB_float = imageB.astype(np.float64)

    MSE = mse_evaluation(imageA_float, imageB_float, fused_image_float)
    PSNR = psnr_evaluation(imageA_float, imageB_float, fused_image_float)
    SSIM = ssim_evaluation(imageA_float, imageB_float, fused_image_float)
    
    return EN, MI, PSNR, MSE, SSIM

# 示例使用



ir_image = read_image(source_image_name1)
vi_image = read_image(source_image_name2)
fused_image = read_image(fused_image_name)

EN, MI, PSNR, MSE, SSIM = analysis_reference(fused_image, ir_image, vi_image, easy=1)

print(f'EN = {EN:.4f}')
print(f'MI = {MI:.4f}')
print(f'PSNR = {PSNR:.4f}')
print(f'MSE = {MSE:.4f}')
print(f'SSIM = {SSIM:.4f}')

 

In [ ]:
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage import io, img_as_float
from scipy.stats import entropy
from skimage.filters import sobel

# 计算熵
def entropy_evaluation(img):
    hist, _ = np.histogram(img, bins=256, range=(0, 255))
    hist_normalized = hist / np.sum(hist)
    ent = entropy(hist_normalized, base=2)
    return ent

# 计算互信息 (Mutual Information)
def mutual_information(hist2d):
    pxy = hist2d / float(np.sum(hist2d))
    px = np.sum(pxy, axis=1)
    py = np.sum(pxy, axis=0)
    px_py = px[:, None] * py[None, :]
    nzs = pxy > 0
    return np.sum(pxy[nzs] * np.log(pxy[nzs] / px_py[nzs]))

def mi_evaluation(img1, img2, img_f):
    hist2d_1 = np.histogram2d(img1.ravel(), img_f.ravel(), bins=256)[0]
    hist2d_2 = np.histogram2d(img2.ravel(), img_f.ravel(), bins=256)[0]
    MI_1 = mutual_information(hist2d_1)
    MI_2 = mutual_information(hist2d_2)
    return MI_1 + MI_2

# 计算 Qabf 指标 (Objective Fusion Performance Measure)
def qabf_evaluation(img1, img2, img_f):
    def sobel_response(img):
        return np.sqrt(sobel(img, axis=0)**2 + sobel(img, axis=1)**2)

    gA = sobel_response(img1)
    gB = sobel_response(img2)
    gF = sobel_response(img_f)

    GAF = np.minimum(gF / gA, gA / gF)
    GAF[gA == 0] = 0
    GAF[gF == 0] = 0

    QgAF = 0.9994 / (1 + np.exp(-15 * (GAF - 0.5)))
    QaAF = 0.9879 / (1 + np.exp(-22 * (1 - abs(GAF - 0.5) * 2 / np.pi)))

    QAF = QgAF * QaAF
    return np.mean(QAF)

# 核心分析函数
def analysis_reference(img_f, img1, img2):
    # 确保图像为灰度并归一化
    img1_float = img_as_float(img1) * 255.0
    img2_float = img_as_float(img2) * 255.0
    img_f_float = img_as_float(img_f) * 255.0
    
    # 计算所需的四个指标
    EN = entropy_evaluation(img_f)
    MI = mi_evaluation(img1, img2, img_f)
    PSNR = psnr(img_f_float, img1_float, data_range=255) + psnr(img_f_float, img2_float, data_range=255)
    Qabf = qabf_evaluation(img1_float, img2_float, img_f_float)

    return EN, MI, PSNR, Qabf

# 示例使用：
if __name__ == '__main__':
    # 加载图像 (假设灰度图像)
    ir_image = io.imread(r'C:\Users\70894\Desktop\LatLRR\IR\18.png', as_gray=True)
    vi_image = io.imread(r'C:\Users\70894\Desktop\LatLRR\VIS\18.png', as_gray=True)
    fused_image = io.imread(r'C:\Users\70894\Desktop\LatLRR\Fused\18.png', as_gray=True)

    EN, MI, PSNR, Qabf = analysis_reference(fused_image, ir_image, vi_image)

    print(f'EN = {EN:.4f}')
    print(f'MI = {MI:.4f}')
    print(f'PSNR = {PSNR:.4f}')
    print(f'Qabf = {Qabf:.4f}')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


# 定义数据
data = {
    'RFN-Nest': [63.63728479, 61.39842746, 64.4649851, 65.89260621, 62.16465553, 66.70638759, 62.59857933, 62.8070123, 63.44198909],
    'PMGI': [63.68306161, 61.28003625, 64.18608113, 64.81266923, 62.53023384, 67.20307299, 64.71867581, 63.85573706, 63.94337857],
    'DenseFuse': [63.62770928, 61.80864972, 64.61324773, 64.91384731, 62.618423, 65.76521922, 62.79544495, 63.42158308, 64.45095434],
    'FusionGAN': [60.13903739, 57.78324483, 59.08493888, 63.13541928, 59.87561574, 63.62682264, 61.81255124, 61.85837283, 62.44329805],
    'our': [64.6524, 61.9843, 65.0121, 66.23121, 63.3423, 66.5678, 63.4526, 63.6823, 63.9946]
}





# 创建DataFrame
df = pd.DataFrame(data)

# 更新X轴数值，长度与数据相同
x_values = range(1, 10)  # X轴数值

# 绘制图像
plt.figure(figsize=(10, 6))

# 绘制每列数据的曲线
for column in df.columns:
    plt.plot(x_values, df[column], label=f'{column}', marker='o')

# 添加标签、标题和图例
plt.title('PSNR', fontsize=14)
plt.xlabel('Proportion', fontsize=12)
plt.ylabel('Values of The Metric', fontsize=12)
plt.legend(title='Method', bbox_to_anchor=(1.05, 1), loc='upper left')

# 显示图像
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np; 
print(np.__version__)
import tensorflow as tf;
print(tf.__version__)
import matplotlib
print(matplotlib.__version__)


